In [1]:
from kliff.dataset import Dataset
from kliff.ml.opt import OptimizerTorch
from kliff.ml import Descriptor
from kliff.ml import TrainingWheels
from kliff.ml.descriptors import get_set51, get_default_bispectrum
import numpy as np
import torch
from torch.nn import Sequential, Linear, ReLU, Tanh
torch.set_default_tensor_type(torch.DoubleTensor)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = Dataset("/home/openkim/MACH2023/data/Si_training_set_4_configs")

2023-04-06 14:44:05.830 | INFO     | kliff.dataset.dataset:_read:687 - 4 configurations read from /home/openkim/MACH2023/data/Si_training_set_4_configs


Provided default set of tested hyperparameters

In [3]:
hyperparameters = get_default_bispectrum()
hyperparameters

OrderedDict([('jmax', 4),
             ('rfac0', 0.99363),
             ('diagonalstyle', 3),
             ('rmin0', 0),
             ('switch_flag', 1),
             ('bzero_flag', 0),
             ('use_shared_array', False),
             ('weights', None)])

### Lets define a simple NN model with descriptor based inputs
The width of descriptor = `desc.width`

In [5]:
desc = Descriptor(species=["Si"], cutoff=3.77, descriptor="Bispectrum",hyperparameters=hyperparameters)
model = Sequential(Linear(desc.width, 10), ReLU(), Linear(10, 10), Tanh(), Linear(10, 1))

### Compute energy and forces
In most ML application, gradients are computed using vector-Jacobian product
i.e. for a neural network mapping the descriptor to energy, and a descriptor function mapping coordinates to descriptor we have:
$$
 \{\mathbf{d}_i\}  = f_{desc}(\{\mathbf{r}_i\})
$$
$$
E_i = f_{energy}({\mathbf{d_i}})
$$
$$
E = \sum_iE_i
$$
then
$$
\mathbf{F} = -\nabla E = -\sum_i\sum_j \frac{d E_i}{d\mathbf{d}_i} \times \frac{d\mathbf{d_i}}{d\mathbf{r}_j}
$$
$$
 = \sum_i \frac{dE}{d\mathbf{d}_i}\times\mathbf{J}_{f_{desc}}
$$

In [6]:
# get descriptor
representation = torch.tensor(desc.forward(dataset[0]))
representation.requires_grad_(True)

# energy
atomwise_energy = model(representation)
total_energy = atomwise_energy.sum()

# dE/d desc
total_energy.backward()
dE_ddesc = representation.grad.detach().numpy()

# dE/dr = dE/d desc x J_Bispectrum
forces = desc.backward(dataset[0], dE_ddesc)
print(forces)

[[ 0.10810825  0.03119557 -0.05810161]
 [-0.05567654 -0.02797805 -0.02375742]
 [ 0.04304471 -0.04291705 -0.08127616]
 [-0.07743129 -0.10008315 -0.04402768]
 [ 0.03399499  0.11126101 -0.01007131]
 [-0.07195698  0.0402178   0.07060787]
 [ 0.04735938 -0.13548004  0.07265382]
 [ 0.09475799  0.03421475  0.07488377]
 [ 0.00355193 -0.12042169 -0.05137739]
 [ 0.03396748  0.0457662   0.11728549]
 [ 0.00639155 -0.00796861  0.00355223]
 [-0.08806111  0.06182617 -0.12313624]
 [ 0.11873553 -0.01642035  0.07720148]
 [-0.0051028  -0.08482003 -0.0426131 ]
 [ 0.04038053  0.06247    -0.10272096]
 [ 0.02608551  0.09763462 -0.03330721]
 [ 0.1220361  -0.0908466  -0.05322691]
 [-0.05194974 -0.09735867 -0.10683447]
 [-0.09709146 -0.05918408  0.12742432]
 [ 0.06836982  0.06243279 -0.14641221]
 [ 0.11179643 -0.06626432  0.00436644]
 [ 0.11107769 -0.01523744  0.13168169]
 [ 0.01297764  0.06266213  0.00495013]
 [ 0.04997557  0.0395551   0.05625087]
 [-0.04394532  0.0452718  -0.102608  ]
 [ 0.03365871  0.0504526 

### Using TrainingWheels

In [8]:
model_tw = TrainingWheels.init_descriptor(model=model, cutoff=3.77, descriptor_kind="Bispectrum", species=["Si"], 
                               hyperparams=hyperparameters, model_returns_forces=False)

In [9]:
model_tw(dataset[0])

{'energy': tensor(-30.0164, grad_fn=<SumBackward0>),
 'forces': tensor([[-0.1081, -0.0312,  0.0581],
         [ 0.0557,  0.0280,  0.0238],
         [-0.0430,  0.0429,  0.0813],
         [ 0.0774,  0.1001,  0.0440],
         [-0.0340, -0.1113,  0.0101],
         [ 0.0720, -0.0402, -0.0706],
         [-0.0474,  0.1355, -0.0727],
         [-0.0948, -0.0342, -0.0749],
         [-0.0036,  0.1204,  0.0514],
         [-0.0340, -0.0458, -0.1173],
         [-0.0064,  0.0080, -0.0036],
         [ 0.0881, -0.0618,  0.1231],
         [-0.1187,  0.0164, -0.0772],
         [ 0.0051,  0.0848,  0.0426],
         [-0.0404, -0.0625,  0.1027],
         [-0.0261, -0.0976,  0.0333],
         [-0.1220,  0.0908,  0.0532],
         [ 0.0519,  0.0974,  0.1068],
         [ 0.0971,  0.0592, -0.1274],
         [-0.0684, -0.0624,  0.1464],
         [-0.1118,  0.0663, -0.0044],
         [-0.1111,  0.0152, -0.1317],
         [-0.0130, -0.0627, -0.0050],
         [-0.0500, -0.0396, -0.0563],
         [ 0.0439, -0.045

### Train model

In [10]:
desc = Descriptor(species=["Si"], cutoff=3.77, descriptor="SymmetryFunctions",hyperparameters=get_set51())
model = Sequential(Linear(desc.width, 10), ReLU(), Linear(10, 10), Tanh(), Linear(10, 1))
model_tw = TrainingWheels.init_descriptor(model=model, cutoff=3.77, descriptor_kind="SymmetryFunctions", species=["Si"], 
                               hyperparams=get_set51(), model_returns_forces=False)

In [11]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
opt = OptimizerTorch(model_tw, dataset, optimizer=optimizer, epochs=5)

In [12]:
_ = opt.minimize()

2023-04-06 14:56:00.799 | INFO     | kliff.ml.opt:minimize:295 - Starting with method Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.01
    maximize: False
    weight_decay: 0
)


In [13]:
model_tw.save_kim_model("ExampleModel")